In [28]:
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import math
import numpy as np

In [29]:
fileName=['a01','a02','a03','a04','a05','a06','a07','a08','a09','a10','a11','a12','a13','a14','a15','a16','a17','a18','a19','a20',
         'b01','b02','b03','b04','b05','c01','c02','c03','c04','c05','c06','c07','c08','c09','c10',
         'x01','x02','x03','x04','x05','x06','x07','x08','x09','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20']
mypath='C:/Users/sara/Documents/5anno/TESI/DataBase_PhysioNet/www.physionet.org/physiobank/database/apnea-ecg/'

In [30]:
df_features_list=[]
for index in range(0,35):
    #index=0
    print(index)
    recordname=mypath+fileName[index]
    record = wfdb.rdsamp(recordname)
    dfECG=pd.DataFrame()
    ecg=record[0]
    Fs_ecg=record[1]['fs']
    intervallo=len(ecg)/(len(ecg)*Fs_ecg)
    time_ecg=[]
    for i in range(0,len(ecg)):
        time_ecg.append(intervallo*i)
    dfECG['time']=time_ecg
    dfECG['ecg']=ecg

    # valori label APNEA
    dfAPNEA=pd.DataFrame()
    ann = wfdb.rdann(recordname, extension="apn")
    time_apn=ann.sample/100
    apn=ann.symbol

    dfAPNEA['time']=time_apn
    dfAPNEA['apnea']=apn

    # valori QRS
    #dfQRS=pd.DataFrame()
    #ann = wfdb.rdann(recordname, extension="qrs")
    #time_qrs=ann.sample/100
    #dfQRS['time']=time_qrs


    #leggo RR 
    dfQRS = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'RR.pkl')
    RR_intervals=dfQRS['rr']

    #leggo EDR
   # dfEDR = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'EDR.pkl')
#edr=dfEDR['edr']

    #divido in epoch di 60 s
    lunghezza_intervalli=60*Fs_ecg
    df=pd.DataFrame();
    startTime=[]
    stopTime=[]
    startRRIndex=[]
    stopRRIndex=[]
  #  startEDRIndex=[]
   # stopEDRIndex=[]

    numIntervalli=len(dfECG)/lunghezza_intervalli
    if(len(ecg)%lunghezza_intervalli!=0):
        numIntervalli+=1
    # print(numIntervalli)
    for i in range(0,math.floor(numIntervalli)-1):
       # mediaECG.append((ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]).mean())
        startTime.append(dfECG['time'][i*lunghezza_intervalli])
        stopTime.append(dfECG['time'][i*lunghezza_intervalli+lunghezza_intervalli-1])
       # devECG.append(np.std(ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]))

    df['startTime']=startTime  
    df['stopTime']=stopTime  
    df['time_apn']=dfAPNEA['time']
    df['label']=dfAPNEA['apnea']

    #------------- DIVISIONE RR IN GRUPPI DI 60 s-------------
    k=0
    for i in range (0, len(df)):
            if(k<len(RR_intervals)):
                startRRIndex.append(k)
                k=k+1
                while( k<len(dfQRS) and df['stopTime'][i]>=dfQRS['time'][k] ):
                    k=k+1
                stopRRIndex.append(k-1)
                #print(k-1)
           # else:
              #  startRRIndex.append(np.nan)
               # stopRRIndex.append(np.nan)
    df['startRRIndex']=startRRIndex
    df['stopRRIndex']=stopRRIndex
    df['startEDRIndex']=startRRIndex
    df['stopEDRIndex']=stopRRIndex
    df=df.dropna()
    df.to_pickle(mypath+'DivisioneDB/'+fileName[index]+'divisione.pkl')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [33]:
df

,startTime,stopTime,time_apn,label,startRRIndex,stopRRIndex,startEDRIndex,stopEDRIndex
0,0.0,59.99,0.0,N,0,0,0,0
1,60.0,119.99,60.0,N,1,1,1,1
2,120.0,179.99,120.0,N,2,2,2,2
3,180.0,239.99,180.0,N,3,3,3,3
4,240.0,299.99,240.0,N,4,25,4,25
5,300.0,359.99,300.0,N,26,40,26,40
6,360.0,419.99,360.0,N,41,41,41,41
7,420.0,479.99,420.0,N,42,42,42,42
8,480.0,539.99,480.0,N,43,47,43,47
9,540.0,599.99,540.0,N,48,56,48,56


In [39]:
dfQRS['time'][25]

295.27